In [10]:
import time
import os
import csv
from PIL import Image
from PIL.ExifTags import TAGS
from keras.models import load_model
from keras.models import model_from_json
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np
import decimal

json_file = open('bladders.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("bladders.h5")

#format of logfile: timestamp, tataoba:yes/no
logfile = []
def image_log_list(image_path):
    log_of_image = []
    ts = time.time()
    log_of_image.append(ts)
    classifier_output = process_image(image_path)
    log_of_image.append(classifier_output)
    exif_data_output = exif_data_check(image_path)
    log_of_image.append(exif_data_output)
    location_data_output = location_data(image_path)
    log_of_image.append(location_data_output)
    return log_of_image
    
def process_image(image_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    if(np.argmax(preds) == 0):
        output = "Tataoba Bladder detected: YES"
    else:
        output = "Tataoba Bladder detected: NO"

    return output

def exif_data_check(image_path):
    image = Image.open(image_path)
    exifdata = image.getexif()
    if len(exifdata) == 0:
        output = "EXIF data: Not available"
    else:
        output = "EXIF data: available"
    return output

def location_data(image_path):
    image = Image.open(image_path)
    exif = image.getexif()
    if len(exif) == 0:
        output = "Location data: Not available"
        return output
    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                output = "Location Data: Not available"
                return output

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    output = str(geotagging)
    return output

def generate_csv(logfile):
    with open('output.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(logfile)

    
input_folder_path = input("Enter file path: ")
for img in os.listdir(input_folder_path):
    if not img.startswith('.'):
        img_path = os.path.join(input_folder_path,img)
        img_log_list = image_log_list(img_path)
        logfile.append(img_log_list)
        
generate_csv(logfile)

print("All processing done, check generated csv for data about processed images")
        
    
    

Enter file path: test/bladder
All processing done, check generated csv for data about processed images
